# Comparación de documentos usando TF-IDF y Word2Vec en Español

Este notebook permite subir dos archivos de texto `.txt`, calcular su similitud usando dos enfoques:
- TF-IDF + Cosine Similarity
- Word2Vec + Cosine Similarity

Incluye comentarios para uso académico y es compatible con Google Colab.

In [ ]:
#!pip install gensim nltk scikit-learn
from google.colab import files
uploaded = files.upload()  # Subir documento1.txt y documento2.txt


Saving documento1.txt to documento1.txt
Saving documento2.txt to documento2.txt


In [ ]:
doc1 = open("documento1.txt", encoding="utf-8").read()
doc2 = open("documento2.txt", encoding="utf-8").read()
print("Documento 1:", doc1[:150], "...")
print("Documento 2:", doc2[:150], "...")

Documento 1: La inteligencia artificial (IA) está revolucionando diversos sectores, desde la salud hasta la educación. 
Las empresas están utilizando IA para autom ...
Documento 2: El cambio climático representa una amenaza global que requiere atención inmediata. 
Los fenómenos naturales extremos son cada vez más frecuentes e int ...


In [ ]:

import re
import nltk
nltk.download("stopwords")
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stopwords_es = set(stopwords.words('spanish'))

def limpiar_texto(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', '', texto)
    tokens = word_tokenize(texto)
    tokens = [t for t in tokens if t not in stopwords_es and len(t) > 2]
    return tokens

tokens1 = limpiar_texto(doc1)
tokens2 = limpiar_texto(doc2)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


## Similitud con TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf = TfidfVectorizer()
X = tfidf.fit_transform([" ".join(tokens1), " ".join(tokens2)])
sim_tfidf = cosine_similarity(X[0:1], X[1:2])
print(f"Similitud TF-IDF: {sim_tfidf[0][0]:.4f}")

Similitud TF-IDF: 0.0562


## Similitud con Word2Vec (modelo preentrenado en español)

In [ ]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: 

In [ ]:
# Paso 2: Cargar el modelo binario (Gensim lee .gz directamente)
from gensim.models import KeyedVectors
import os

# Nombre real del archivo que bajaste
SBW_PATH = "SBW-vectors-300-min5.bin.gz"

# (Opcional) limita vocab para ahorrar RAM; None = carga todo (~1GB+)
LOAD_LIMIT = None  # por ejemplo 500_000 si te quedas corto de memoria

assert os.path.exists(SBW_PATH), f"No encuentro {SBW_PATH} en el directorio actual."

# Carga robusta
modelo = KeyedVectors.load_word2vec_format(
    SBW_PATH,
    binary=True,
    unicode_errors="ignore",
    limit=LOAD_LIMIT
)

print("Embeddings cargados:",
      f"{len(modelo.index_to_key):,} tokens • dimensión:", modelo.vector_size)

# (Opcional recomendado): guarda en formato .kv para cargas rápidas futuras
# modelo.save("SBW-vectors-300-min5.kv")


Embeddings cargados: 1,000,653 tokens • dimensión: 300


In [ ]:
# Paso 4: Preprocesar los documentos
import re
import unicodedata

# 1) La tuya (elimina acentos y símbolos)
def limpiar_texto_sin_acentos(texto):
    texto = texto.lower()
    texto = unicodedata.normalize('NFD', texto).encode('ascii', 'ignore').decode("utf-8")
    texto = re.sub(r'[^\w\s]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto.split()

# 2) Alternativa (preserva acentos y ñ, solo limpia puntuación)
def limpiar_texto_con_acentos(texto):
    texto = texto.lower()
    # conserva acentos/ñ; solo quita puntuación rara
    texto = re.sub(r'[^\w\sáéíóúüñ]', ' ', texto, flags=re.IGNORECASE)
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto.split()

# Elige la que prefieras:
limpiar_texto = limpiar_texto_con_acentos  # o limpiar_texto_sin_acentos

tokens1 = limpiar_texto(doc1)
tokens2 = limpiar_texto(doc2)
print("Tokens documento 1:", tokens1[:30], "...")
print("Tokens documento 2:", tokens2[:30], "...")


Tokens documento 1: ['la', 'inteligencia', 'artificial', 'ia', 'está', 'revolucionando', 'diversos', 'sectores', 'desde', 'la', 'salud', 'hasta', 'la', 'educación', 'las', 'empresas', 'están', 'utilizando', 'ia', 'para', 'automatizar', 'procesos', 'y', 'mejorar', 'la', 'eficiencia', 'una', 'de', 'las', 'aplicaciones'] ...
Tokens documento 2: ['el', 'cambio', 'climático', 'representa', 'una', 'amenaza', 'global', 'que', 'requiere', 'atención', 'inmediata', 'los', 'fenómenos', 'naturales', 'extremos', 'son', 'cada', 'vez', 'más', 'frecuentes', 'e', 'intensos', 'las', 'emisiones', 'de', 'gases', 'de', 'efecto', 'invernadero', 'son'] ...


In [ ]:
# Cargar SBW (español)
from gensim.models import KeyedVectors
kv = KeyedVectors.load_word2vec_format("SBW-vectors-300-min5.bin.gz", binary=True)

import re, math, numpy as np
from collections import Counter

# Stopwords básicas en español (puedes ampliar)
STOP_ES = set("""
a al algo algun alguna algunas alguno algunos ante antes aquel aquella aquellas aquellos aqui asi aunque
bajo bien
cada como con contra cual cuales cuando de del desde donde dos durante
e el ella ellas ello ellos en entre era erais eramos eran eres es esa esas ese eso esos esta estaba estaban
estamos estan estar este esto estos fue fueron fui fuimos ha haber habia habian hago hace hacen hacer hacia
han hasta hay la las le les lo los mas me mi mis mucho muy nada ni no nos nosotros nuestra nuestras nuestro
nuestros o os otra otras otro otros para pero poco por porque que quien quienes se ser si sino sin sobre solo
son soy su sus tambien tan tanto te tiene tienen toda todas todo todos tras tu tus un una unas uno unos
usted ustedes ya
""".split())

def tokenize_preservando_acentos(s: str):
    s = s.lower()
    # conserva áéíóúüñ y letras/números, reemplaza lo demás por espacio
    s = re.sub(r"[^\w\sáéíóúüñ]", " ", s, flags=re.IGNORECASE)
    s = re.sub(r"\s+", " ", s).strip()
    return s.split()

def build_idf(texts_tokens):
    N = len(texts_tokens)
    df = Counter()
    for toks in texts_tokens:
        df.update(set(toks))
    # IDF suavizado
    return {w: math.log((N + 1) / (df[w] + 1)) + 1.0 for w in df}

def doc_embedding(tokens, kv, idf=None):
    vecs = []
    weights = []
    for t in tokens:
        if t in STOP_ES:
            continue
        if t in kv.key_to_index:
            w = idf.get(t, 1.0) if idf else 1.0
            vecs.append(kv[t] * w)
            weights.append(w)
    if not vecs:
        return np.zeros(kv.vector_size, dtype=np.float32)
    V = np.vstack(vecs)
    W = np.array(weights, dtype=np.float32).reshape(-1,1)
    v = V.sum(axis=0) / (W.sum())
    # normaliza (importante para coseno)
    n = np.linalg.norm(v) + 1e-8
    return (v / n).astype(np.float32)

def coseno(a, b):
    return float(np.dot(a, b))

# ---- Usa tus doc1/doc2 aquí ----
toks1 = tokenize_preservando_acentos(doc1)
toks2 = tokenize_preservando_acentos(doc2)

# IDF sobre ambos (o mejor, sobre varios docs si tienes)
idf = build_idf([toks1, toks2])

v1 = doc_embedding(toks1, kv, idf=idf)
v2 = doc_embedding(toks2, kv, idf=idf)
print("Similitud coseno (IDF + sin stopwords):", round(coseno(v1, v2), 4))


ModuleNotFoundError: No module named 'gensim'